In [1]:
import pickle
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import math

# Helpers
from testing import *
# Baseline model 1
from transformers import RobertaTokenizer, RobertaForMaskedLM, pipeline
# Sentence similarity model
# https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
from sentence_transformers import SentenceTransformer

# Check if GPU acceleration is available
if torch.cuda.is_available():
    device = "cuda"
    device_num = torch.cuda.current_device()
else:
    device = "cpu"
    device_num = -1

print(f"Using {device} device")

Using cuda device


In [2]:
# Run this if you removed pickle files
# saving_pickles()

train_df, valid_df, test_df = loading_pickles()

In [23]:
# Constants (for training) -------------------------------------------------------------------------------------------------

mask_prob = 0.5
window_size = 100
# For training (not validation)
batch_size = 50

# Whether to shuffle the training set or not
shuffle = False
# Changing this may lead to different masking result. Do not change unless necessary.
rng_seed = 42

# Using only a portion of the train dataset for performance
# Set it to 1 to use all train dataset; increasing this may give better result.
size_limit_proportion_train = 0.01

# For validation set. Do not change unless necessary
size_limit_proportion_valid = 0.5

# Hyperparameters ----------------------------------------------------------------------------------------------------------

total_epoch_number = 2
# Initial learning rate for the optimizer
learning_rate = 0.00005
weight_decay_coefficient = 0.01

In [16]:
def masking_df(code_df):
    masked_code_df = mask_variable_df(code_df, mask_prob=mask_prob, rng_seed=rng_seed)
    merged_code_df = pd.concat([code_df, masked_code_df], axis="columns")
    return merged_code_df

def window_df(code_df):
    merged_code_df = masking_df(code_df)
    return split_into_windows(merged_code_df, window_size=window_size, mask_token="<mask>")

train_df_size = int(len(train_df) * size_limit_proportion_train)
valid_df_size = int(len(valid_df) * size_limit_proportion_valid)

if shuffle:
    train_df = train_df.sample(frac=1, random_state=rng_seed).reset_index(drop=True)

window_train_df = window_df(train_df[:train_df_size])

merged_valid_df = masking_df(valid_df[:valid_df_size])
merged_test_df = masking_df(test_df)

Masking:   0%|          | 0/4121 [00:00<?, ?it/s]

Window split:   0%|          | 0/4121 [00:00<?, ?it/s]

Masking:   0%|          | 0/23107 [00:00<?, ?it/s]

Masking:   0%|          | 0/22176 [00:00<?, ?it/s]

three ways to fine tune the model:
1. Fine-tune the last head of the model <- Current approach
2. Fine-tune the whole model
3. Add extra layers to the model and fine tune the additional layer

We use the first baseline model, as it showed the better performance in testing.
See the baseline_testing notebook for more details.

There are three main hyperparameters: total_epoch_number, learning_rate, and weight_decay_coefficient
Try different combinations to fine the best performing model. I commented thoroughly so there should be no problem in understanding the codes.
If you want to change the optimizer, then you need to directly edit the optimizer definition in the training loop cell.

After a model is fine-tuned, its performance will then be evaluated based on the average cosine similarity with the validaion set.
You can check if the fine-tuned model performs better or worse compared to the baseline model. If your model performs well, then
consider saving the model so that we can use it later. For performance, it is set to only use the half of the validation set.

If any error occurs, or if you have a suggestion, let me know.


### Model investigation

You need to uncomment some cells to see the results.

In [5]:
# All Huggingface models are standard torch.nn.Module, so they can easily be used in any training loop.

# Model architecture information:
# https://huggingface.co/docs/transformers/v4.27.2/en/model_doc/roberta#transformers.RobertaForMaskedLM
model_b1 = RobertaForMaskedLM.from_pretrained('microsoft/codebert-base-mlm')
tokenizer_b1 = RobertaTokenizer.from_pretrained('microsoft/codebert-base-mlm')

In [6]:
# Model structure (Uncomment to see)
# print(model_b1)

# Embedding size = (50265, 768)
# Dropout probability = 0.1

In [7]:
# Model parameters (Uncomment to see)

# for name, param in model_b1.named_parameters():
#     print(name)

In [8]:
# Freeze parameters except the last head

for name, param in model_b1.named_parameters():
    if "lm_head" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

    print(name, param.requires_grad)

roberta.embeddings.word_embeddings.weight False
roberta.embeddings.position_embeddings.weight False
roberta.embeddings.token_type_embeddings.weight False
roberta.embeddings.LayerNorm.weight False
roberta.embeddings.LayerNorm.bias False
roberta.encoder.layer.0.attention.self.query.weight False
roberta.encoder.layer.0.attention.self.query.bias False
roberta.encoder.layer.0.attention.self.key.weight False
roberta.encoder.layer.0.attention.self.key.bias False
roberta.encoder.layer.0.attention.self.value.weight False
roberta.encoder.layer.0.attention.self.value.bias False
roberta.encoder.layer.0.attention.output.dense.weight False
roberta.encoder.layer.0.attention.output.dense.bias False
roberta.encoder.layer.0.attention.output.LayerNorm.weight False
roberta.encoder.layer.0.attention.output.LayerNorm.bias False
roberta.encoder.layer.0.intermediate.dense.weight False
roberta.encoder.layer.0.intermediate.dense.bias False
roberta.encoder.layer.0.output.dense.weight False
roberta.encoder.layer.

### Preparing the dataset

In [9]:
class CodeNetDataset(Dataset):
    def __init__(self, window_df, tokenizer):
        self.window_df = window_df
        self.tokenizer = tokenizer
        self.mask_token_id = tokenizer.mask_token_id

        # Not sure how to apply tqdm (progress bar) for this; I plan to update soon
        self.tokenized = tokenizer(list(window_df["window"]), padding=True)
        self.input_ids = self.tokenized["input_ids"]
        self.attention_mask = self.tokenized["attention_mask"]
        self.label = self.window_df["label"]

    def __len__(self):
        return len(self.window_df)

    def __getitem__(self, index):
        label_token_id = self.tokenizer.convert_tokens_to_ids(self.label[index])
        # size is equal to the vocabulary size
        one_hot_label = [0] * self.tokenizer.vocab_size
        one_hot_label[label_token_id] = 1
        mask_token_index = self.input_ids[index].index(self.mask_token_id)

        return torch.tensor(self.input_ids[index]), torch.tensor(self.attention_mask[index]), torch.tensor(one_hot_label), mask_token_index

train_dataset = CodeNetDataset(window_train_df, tokenizer_b1)
valid_dataset = CodeNetDataset(window_valid_df, tokenizer_b1)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
# batch_size = dataset_size (to get the entire dataset at once)
valid_dataloader = DataLoader(valid_dataset, batch_size=valid_dataset.__len__())

### Fine-tuning

As we are using another sentence embedding model for calculating the cosine similarity, the loss value needs to be backpropagated through the embedding model as well. But it seems infeasible for this moment, so I will going to implement the loss function only on the cross-entropy (perplexity). Also, we only use the top_1 prediction as it showed the best performance in the baseline testing.

In [10]:
# Loss function (Softmax + CrossEntropy)
# The result values from the model is logits, which cannot be compared with one-hot true labels.
# For example, logits varies from the negative infinity to the positive infinity, and the positive infinity is equal to the 100% probability.
# But the one-hot true labels has only 0 and 1, and 1 means 100% probability.
# So the Softmax will be applied on the logits, before they are compared by the CrossEntropy.
def soft_entropy(prediction_logits, one_hot_labels, mask_token_indices):
    # Getting the embeddings only for the mask token locations
    mask_embedding_list = list()
    for row, mask_token_index in zip(range(prediction_logits.shape[0]), mask_token_indices):
        # The sliced tensors are single-dimensional vectors. So we need to add a dummy dimension at dim=0
        # so that they can be concatenated in dim=0
        # For example, if the sliced tensors have the shape [5], torch.unsqueeze() makes it to [1, 5]
        mask_embedding = torch.unsqueeze(prediction_logits[row, mask_token_index], 0)
        mask_embedding_list.append(mask_embedding)

    # Shape = [batch_size, vocabulary_size]
    mask_embeddings = torch.cat(mask_embedding_list, dim=0)
    probabilities = F.softmax(mask_embeddings, dim=1)

    loss = torch.nn.CrossEntropyLoss()
    # By default, it returns a single scalar (averaged over batch)
    return loss(probabilities, one_hot_labels.float())

In [11]:
# Running this cell multiple times in a single notebook can fully saturate the GPU memory, which leads the OutOfMemoryError.
# If it happens, re-start the notebook kernel to remove all model instances from the GPU memory.
# It seems like the memory error sometimes happens when the dataset size is too big, regardless of the batch size.
# I am not sure why it is the case; Let me know if you encounter this issue.

# Setting the model to the train mode
model_b1.train()
model_b1.to(device)

# Using the AdamW optimizer: https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html
# Feel free to try others
optimizer = torch.optim.AdamW(model_b1.parameters(), lr=learning_rate, weight_decay=weight_decay_coefficient)

for epoch_num in range(1, total_epoch_number+1):
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch_num}", total=math.ceil(len(train_dataset) / batch_size)):
        # Sending to GPU
        batch_input_ids = batch[0].to(device)
        batch_attention_mask = batch[1].to(device)
        batch_one_hot_label = batch[2].to(device)
        batch_mask_token_index = batch[3].to(device)

        # Forward pass
        prediction_logits = model_b1(input_ids=batch_input_ids, attention_mask=batch_attention_mask).logits
        # Computing prediction error
        loss = soft_entropy(prediction_logits, batch_one_hot_label, batch_mask_token_index)
        # Removing gradients from the past iteration
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Removing to free up memory
        del batch_input_ids
        del batch_attention_mask
        del batch_one_hot_label
        del batch_mask_token_index
        torch.cuda.empty_cache()

    print(f"Epoch {epoch_num}, Loss: {loss.item()}")

# Setting the model back to the evaluation mode
model_b1.eval()
# Not to print the model structure
print()

Epoch 1:   0%|          | 0/246 [00:00<?, ?it/s]

Epoch 1, Loss: 10.745734214782715


Epoch 2:   0%|          | 0/246 [00:00<?, ?it/s]

Epoch 2, Loss: 10.72003173828125


RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

### Validation (Cosine similarity)

In [18]:
b1_finetuned = pipeline('fill-mask', model=model_b1, tokenizer=tokenizer_b1, device=device_num)
b1_finetuned_result = model_test(merged_code_df=merged_valid_df, unmasker=b1_finetuned, top_k=1, window_size=window_size, batch_size=batch_size)

Window split:   0%|          | 0/23107 [00:00<?, ?it/s]

Prediction:   0%|          | 0/67804 [00:00<?, ?it/s]

Similarity:   0%|          | 0/67804 [00:00<?, ?it/s]

In [ ]:
torch.cuda.empty_cache()

In [19]:
average_similarity = np.mean(b1_finetuned_result['similarity'])
# Baseline performance (I plan to send you the result by this evening)
baseline_similarity = to_be_confirmed
print(f"Average cosine similarity: {average_similarity}")
if average_similarity < baseline_similarity:
    print(f"The fine-tuned model performance is worse than the baseline, by {baseline_similarity-average_similarity}")
else:
    print(f"The fine-tuned model performance is better than the baseline, by {average_similarity-baseline_similarity}")

Average cosine similarity: 0.49379345774650574
The fine-tuned model performance is better than the baseline, by 0.01018860936164856


### Saving the model

Check the following for loading the model: https://pytorch.org/docs/stable/generated/torch.load.html#torch.load

In [24]:
# Saving the hyperparameters and the fine-tuned model
# The model size is around 500 MB
training_constants = [mask_prob, window_size, batch_size, total_epoch_number, learning_rate, weight_decay_coefficient, shuffle, rng_seed, size_limit_proportion_train, size_limit_proportion_valid]
result_dict = {"training_constants": training_constants, "validation_similarity": average_similarity}

In [25]:
# Generating non-existing filenames
file_index = 0
while os.path.exists(f"./saved_models/model_{file_index}"):
    file_index += 1

model_filepath = f"./saved_models/model_{file_index}"
hparameter_filepath = f"./saved_models/hparameter_{file_index}"

In [26]:
# Saving the model
torch.save(model_b1, model_filepath)
# Saving the hyperparameters
with open(hparameter_filepath, "wb") as fw:
    pickle.dump(result_dict, fw)